In [2]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense

import numpy as np
import matplotlib.pyplot as plt

import tensorflow.keras.backend as K


In [3]:
## TODO: train our own neural network

In [4]:
## use for pre-trained neural network
K.clear_session()
pre_trained = 'vgg16'
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image

nrow = 32
ncol = 32
input_shape = (nrow,ncol,3)
base_model = applications.VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
model = Sequential()
for layer in base_model.layers:
    model.add(layer)
for layer in model.layers:
    layer.trainable = False
model.add(Flatten())
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

NameError: name 'applications' is not defined